In [1]:
from config import g_key
import gmaps
import pandas as pd
import requests

In [2]:
cities_file_path = "../WeatherPY/cities_data.csv"

# Import the csv file using pandas
cities_df = pd.read_csv(cities_file_path)

cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,0,Longaví,81,CL,1601721057,81,-35.97,-71.68,4.99,1.57
1,1,Provideniya,90,RU,1601721058,83,64.38,-173.30,5.30,5.36
2,2,Albany,80,US,1601721027,86,42.60,-73.97,7.22,1.33
3,3,Dikson,100,RU,1601721059,88,73.51,80.55,1.26,10.65
4,4,George Town,20,MY,1601720818,70,5.41,100.34,30.00,3.60


In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_df[["Lat", "Lon"]]
humidity = cities_df["Humidity"].astype(float)

figure = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=True, max_intensity=50, point_radius=2.5)
figure.add_layer(heat_layer)

figure

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Creating new df based off of the orginal
conditions_df = cities_df.copy()
conditions_df = conditions_df[
    (conditions_df['Max Temp'] >= 21.111) & 
    (conditions_df['Max Temp'] <= 26.666) & 
    (conditions_df['Wind Speed'] <= 16.06) & 
    (conditions_df['Cloudiness'] == 0)
]

# Drop nulls and reset index
conditions_df = conditions_df.dropna()
conditions_df = conditions_df.reset_index()

# Drop any additional columns from your data set
conditions_df_drop = conditions_df.drop(['index', 'Unnamed: 0'], axis=1)
conditions_df_drop

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,Todos Santos,0,MX,1601721146,81,23.45,-110.22,25.00,0.45
1,São João da Barra,0,BR,1601721166,88,-21.64,-41.05,24.00,4.10
2,Cabo San Lucas,0,MX,1601721176,77,22.89,-109.91,25.00,0.45
3,Prainha,0,BR,1601721233,71,-1.80,-53.48,26.64,2.49
4,Saryagash,0,KZ,1601721280,19,41.45,69.17,23.00,3.10
5,La Cruz,0,MX,1601721301,95,23.92,-106.90,25.00,1.34
6,Samarkand,0,UZ,1601721384,21,39.65,66.96,22.00,3.10
7,Yangi Marg`ilon,0,UZ,1601721384,18,40.43,71.72,22.00,3.10
8,Nouadhibou,0,MR,1601721386,69,20.93,-17.03,23.00,11.80
9,Beyneu,0,KZ,1601721409,18,45.32,55.20,22.61,2.90


In [6]:
hotel_df = pd.DataFrame(conditions_df_drop, columns=["City", "Country", "Lat", "Lon", "Hotel Name"])

hotel_df

,City,Country,Lat,Lon,Hotel Name
0,Todos Santos,MX,23.45,-110.22,NaN
1,São João da Barra,BR,-21.64,-41.05,NaN
2,Cabo San Lucas,MX,22.89,-109.91,NaN
3,Prainha,BR,-1.80,-53.48,NaN
4,Saryagash,KZ,41.45,69.17,NaN
5,La Cruz,MX,23.92,-106.90,NaN
6,Samarkand,UZ,39.65,66.96,NaN
7,Yangi Marg`ilon,UZ,40.43,71.72,NaN
8,Nouadhibou,MR,20.93,-17.03,NaN
9,Beyneu,KZ,45.32,55.20,NaN


In [7]:
# Params dictionary for the requests
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Iterate each row
for index, row in hotel_df.iterrows():
    # Get lat, lng for each row
    lat = row["Lat"]
    lng = row["Lon"]

    # Add 'location' to params
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request
    name_address = requests.get(base_url, params=params)
    
    # Convert to json
    name_address = name_address.json()
    
    # Try update df hotel name. Go on otherwise <--- will leave value as NaN
    try:
        hotel_df.iloc[index, 4] = name_address['results'][0]['name']
    except IndexError:
        continue
        



In [8]:
hotel_df = hotel_df.dropna().reset_index(drop=True)
hotel_df

,City,Country,Lat,Lon,Hotel Name
0,Todos Santos,MX,23.45,-110.22,Hotel San Cristóbal
1,São João da Barra,BR,-21.64,-41.05,OYO Pousada Mediterrâneo
2,Cabo San Lucas,MX,22.89,-109.91,Waldorf Astoria Los Cabos Pedregal
3,Prainha,BR,-1.80,-53.48,Hotel Ágape
4,La Cruz,MX,23.92,-106.90,Hotel Korita
5,Samarkand,UZ,39.65,66.96,Alexander Hotel
6,Yangi Marg`ilon,UZ,40.43,71.72,ATLAS HOTEL
7,Nouadhibou,MR,20.93,-17.03,Free Zone Hotel Nouadhibou
8,Beyneu,KZ,45.32,55.20,"HOTEL ""AKNUR"""
9,Coahuayana Viejo,MX,18.73,-103.68,Hotel Posada Navideña


In [9]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]



In [10]:


markers = gmaps.marker_layer(locations, info_box_content=hotel_info)


locations = cities_df[["Lat", "Lon"]]
humidity = cities_df["Humidity"].astype(float)

figure = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=True, max_intensity=50, point_radius=2.5)
figure.add_layer(heat_layer)
figure.add_layer(markers)
figure

Figure(layout=FigureLayout(height='420px'))